<a href="https://colab.research.google.com/github/kevinkhang2909/ML-learning-journey/blob/main/nlp/markov_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import os
import re
import string
import random
from tqdm import tqdm
from pathlib import Path
from zipfile import ZipFile
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
file_name = '/content/drive/MyDrive/Colab Notebooks/data/sherlock.zip'
  
with ZipFile(file_name, 'r') as zip:
    zip.extractall('/content/')

In [4]:
story_path = Path('/content/sherlock/sherlock')
files = list(story_path.glob('*.txt'))

stories = []
for file in tqdm(files):
    with open(file) as f:
        for line in f:
            line = line.strip()
            if line == '----------': 
                break
            if line != '': 
                stories.append(line)

print(len(stories))
stories[:5]

100%|██████████| 67/67 [00:00<00:00, 295.01it/s]

215021


['THE ADVENTURE OF THE BLUE CARBUNCLE',
 'Arthur Conan Doyle',
 'I had called upon my friend Sherlock Holmes upon the second morning',
 'after Christmas, with the intention of wishing him the compliments of',
 'the season. He was lounging upon the sofa in a purple dressing-gown,']

In [5]:
cleaned_stories = []
for line in tqdm(stories):
    line = line.lower()
    line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
    tokens = word_tokenize(line)
    words = [word for word in tokens if word.isalpha()]
    cleaned_stories += words

print(len(cleaned_stories))

100%|██████████| 215021/215021 [00:36<00:00, 5903.78it/s]

2332247


In [6]:
cleaned_stories[:10]

['the',
 'adventure',
 'of',
 'the',
 'blue',
 'carbuncle',
 'arthur',
 'conan',
 'doyle',
 'i']

In [7]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in tqdm(range(len(cleaned_stories)-n_gram-1), desc='token to nested dict'):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i + j] + " "
            next_state += cleaned_stories[i + j + n_gram] + " "
        curr_state = curr_state[:-1]  # remove space
        next_state = next_state[:-1]  # remove space
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    for curr_state, transition in tqdm(markov_model.items(), desc='prob'):
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model

In [8]:
markov_model = make_markov_model(cleaned_stories)

prob: 100%|██████████| 208717/208717 [00:00<00:00, 483717.93it/s]


In [9]:
print(f'number of states = {len(markov_model.keys())}')
print(markov_model['the game'])

number of states = 208717
{'for the': 0.036036036036036036, 'is up': 0.06306306306306306, 'is and': 0.036036036036036036, 'was afoot': 0.036036036036036036, 'would have': 0.036036036036036036, 'in their': 0.036036036036036036, 'your letter': 0.02702702702702703, 'for all': 0.06306306306306306, 'i am': 0.02702702702702703, 'now count': 0.02702702702702703, 'worth it': 0.02702702702702703, 'you are': 0.02702702702702703, 'my own': 0.02702702702702703, 'at any': 0.02702702702702703, 'mr holmes': 0.02702702702702703, 'ay whats': 0.02702702702702703, 'my friend': 0.02702702702702703, 'fairly by': 0.02702702702702703, 'is not': 0.02702702702702703, 'was not': 0.02702702702702703, 'was in': 0.02702702702702703, 'is hardly': 0.02702702702702703, 'was whist': 0.036036036036036036, 'was up': 0.09009009009009009, 'in that': 0.036036036036036036, 'the lack': 0.036036036036036036, 'is afoot': 0.036036036036036036, 'may wander': 0.02702702702702703, 'now a': 0.02702702702702703}


In [10]:
def generate_story(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story += curr_state + " "
    while n < limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))
        curr_state = next_state[0]
        story += curr_state + " "
        n += 1
    return story

In [11]:
for i in range(20):
    print(f"{i}. {generate_story(markov_model, start='dear holmes', limit=8)}")

0. dear holmes what do you think the quarrel between the hours of the night i am staying with 
1. dear holmes i have my dooties just the one candle that was my singular interview with the lady 
2. dear holmes i fear that you were never in the least if the man or men who have 
3. dear holmes he has not been for my attendance when i arrived at baker street but i can 
4. dear holmes that i often wonder it can not be bought he will get our work done before 
5. dear holmes what do you imagine i am telling you the story is said that there are business 
6. dear holmes i exclaimed i could not tell you as man to fall foul of see here he 
7. dear holmes that i should be here he kept his distance until the curve of the address of 
8. dear holmes said i as i passed his handkerchief over his brow brother morris and may they not 
9. dear holmes you are enough to find out how could i leave the tray form the message and 
10. dear holmes oh yes he is ever ready with a sinister smile the lights of the s

In [12]:
generate_story(markov_model, start='the case', limit=100)

'the case which promise to make the matter at what hour was not long for this affair has been most deplorably handled i feel that the lady had hurriedly ransacked them before deciding that question i trust that they do not always sufficient and you can remember tadpole phelps who was in vain that i asked the worst what can it mean it means disgrace as well but he was at work and yet i thought you had then been a pause the candle held against the glass work in afghanistan coming on the father of this lad tend to some end or else secure her rights would change from his dressing gown and exposed as high as the elbow where you are and where to hide to get you i know what is that holmes the finding of the bride lord st simon had by sheer horror and exhaustion there was a savage wild animal loose about the place belonged to prosaic and yet as clearly as far as it goes that is her ring it may be a marriage between us james and i am in touch with every development upon the high road and when m